### 할로윈 투자 전략
- 11월의 첫날에 구매 -> 4월 마지막날에 판매
- 11월의 첫날에 시작가격을 구매
- 4월 마지막 날에 종가로 판매
- 판매한 금액 / 구매한 금액
- 누적 수익율

In [ ]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [ ]:
# AMZN 데이터 로드 
df = pd.read_csv("../../csv/AMZN.csv", index_col='Date')

In [ ]:
df.head()

In [ ]:
# index를 시계열데이터로 변경 
df.index = pd.to_datetime(df.index)

In [ ]:
# 2000년 11월 데이터를 출력하려면?
df.loc['2000-11'].iloc[0, 0]

In [ ]:
# 2001년 4월의 마지막날의 종가
df.loc['2001-04'].iloc[-1]['Close']

In [ ]:
rtn_list = []
for i in range(2000, 2011, 1):
    # 구매를 하는 월
    buy_mon = f"{i}-11"
    sell_mon = f"{i+1}-4"
    # 구매가 
    buy = df.loc[buy_mon].iloc[0, 0]
    # 판매가 
    sell = df.loc[sell_mon].iloc[-1]['Close']
    rtn_list.append(sell/buy)

In [ ]:
acc_rtn = 1
for i in rtn_list:
    acc_rtn *= i
acc_rtn

In [ ]:
# 시계열데이터에서 시간을 합 
start = datetime(year = 2000, month=11, day=1)
start

In [ ]:
# 5개월 뒤 
start + relativedelta(months=5)

In [ ]:
# 시계열데이터를 문자열로 변경하는 함수? strftime('%Y-%m')
halloween_df = pd.DataFrame()

for i in range(2000, 2011):
    start = datetime(year = i, month=11, day=1)
    end = start + relativedelta(months=5)
    # 시계열을 문자열로
    start = start.strftime('%Y-%m')
    end = end.strftime('%Y-%m')
    # start의 첫번째 인덱스 
    start_df = df.loc[start].head(1)
    end_df = df.loc[end].tail(1)
    halloween_df = pd.concat( [halloween_df, start_df, end_df] )

In [ ]:
halloween_df = halloween_df[['Open', 'Close']]

In [ ]:
halloween_df

In [ ]:
# 수익율 계산 
halloween_df['rtn'] = 1

# 반복문을 이용하여 수익율 계산
# 인덱스의 위치가 홀수인 경우에만 수익율이 계산
for i in range(1, len(halloween_df), 2):
    # print(i)
    # 구매한 가격
    buy = halloween_df.iloc[i-1]['Open']
    sell = halloween_df.iloc[i]['Close']
    rtn = sell / buy
    # print(rtn)
    halloween_df.iloc[i, 2] = rtn

In [ ]:
halloween_df['acc_rtn'] = halloween_df['rtn'].cumprod()

In [ ]:
halloween_df.tail(1)

### 할로윈 투자전략 함수화
- 매개변수 4개 
    - 데이터(_df)
    - 시작년도(_start = 2010)
    - 종료년도(_end = datetime.now().yaer)
    - 월(_mon= 11)
- 복사본 생성 
- 컬럼에 Date가 존재한다면 Date를 인덱스로 변환
- 인덱스를 시계열 데이터로 변경
- 빈 데이터프레임 생성 
- 반복문을 이용하여 시작년도부터 종료 년도까지 반복
    - 시작년도의 월을 포함한 시계열 데이터 생성 (buy_mon)
    - 위에서 만들어진 시계열에서 5개월 뒤 라는 시계열 데이터 생성(sell_mon)
    - buy_mon, sell_mon 시계열데이터에서 년-월의 포멧으로 문자열로 변경
    - buy_mon으로 인덱스를 필터해서 첫번째 인덱스를 추출
    - sell_mon으로 인덱스를 필터해서 마지막 인덱스를 추출
    - 나온 결과를 빈 데이터프레임에 단순 행 결합
- 수익율 계산
- 누적수익율 계산
- 만들어진 데이터프레임과 총 누적수익율을 되돌려준다. 

In [33]:
def halloween(
        _df, 
        _start = 2010, 
        _end = datetime.now().year, 
        _mon = 11
):
    df = _df.copy()
    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)
    # 인덱스를 시계열로 변경 
    df.index = pd.to_datetime(df.index, utc=True)

    # 비어있는 데이터프레임을 생성
    result = pd.DataFrame()

    # 반복문을 이용해서 _start, _end까지 반복실행하는 반복문
    for i in  range(_start, _end):
        buy_mon = datetime(year = i, month= _mon, day=1)
        sell_mon = buy_mon + relativedelta(months=5)
        buy_mon = buy_mon.strftime('%Y-%m')
        sell_mon = sell_mon.strftime('%Y-%m')
        try:
            buy = df.loc[buy_mon].head(1)
            sell = df.loc[sell_mon].tail(1)
            result = pd.concat( [result, buy, sell] , axis=0)
        except:
            break
    # 수익율 계산
    result['rtn'] = 1
    for idx in range(1, len(result), 2):
        rtn = result.iloc[idx]['Close'] / result.iloc[idx-1]['Open']
        result.iloc[idx, -1] = rtn
    # 누적 수익율 계산 
    result['acc_rtn'] = result['rtn'].cumprod()
    # 최종 누적 수익율 변수에 저장 
    acc_rtn = result.iloc[-1, -1]
    return result, acc_rtn


In [47]:
df = pd.read_csv("../../csv/AMZN.csv", index_col='Date')

In [ ]:
halloween(df, _mon=2)

In [48]:
rtn_data = 0
for i in range(1, 13):
    hw_df, acc_rtn = halloween(df, _mon = i)
    # print(f"{i}월 부터 6개월 투자한 수익율 : {acc_rtn}")
    if acc_rtn > rtn_data:
        rtn_data = acc_rtn
        month_data = i
print(f"{month_data}월에 6개월 투자 수익율이 {rtn_data}입니다.")

4월에 6개월 투자 수익율이 5.894513902549153입니다.


C:\Users\moons\AppData\Local\Temp\ipykernel_16824\728782376.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.8019082715596331' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn
C:\Users\moons\AppData\Local\Temp\ipykernel_16824\728782376.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9570547085565838' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn
C:\Users\moons\AppData\Local\Temp\ipykernel_16824\728782376.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '1.051642840395354' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn
C:\Users\moons\AppData\Local\Temp\ipyk

9